In [2]:
import pandas as pd
import numpy as np
import glob
import os
import nilearn.signal as nls
import matplotlib.pyplot as plt
from scipy.stats import zscore as zscore

In [16]:
#task names
tasks=['3T_tfMRI_FACENAME', '3T_tfMRI_VISMOTOR', '3T_tfMRI_CARIT'] 

#Glasser index for tables
gla_index = np.loadtxt('/media/DataD800/Alina/atlases/gla_index.txt', dtype=str)

#loop through the task
for task in tasks:

    #specify the path to data
    path = '/media/data/HCPAging/data/'+task+'/data/' 

    #read subject names there
    subjects=[i.split('/')[-1] for i in  sorted(glob.glob(path+'*_V1_MR'))]

    #dictionary to store group results
    dct_group_fc={}

    #loop though the subjects
    for subject in subjects:

        ### Load cortex series
        dfcort = pd.read_csv(glob.glob(path+subject+'/signalsROI/cort_MSMAll_av_Glasser_txt/'+subject+'*')[0], 
                             index_col=None, header=None, delimiter='\t')
        dfcort.index=gla_index
        dfcort = dfcort.T

        ### Load subcortex series
        dct_subc={}
        files = sorted(glob.glob(path+subject+'/signalsROI/subc_FS_MSMAll_txts/'+subject+'/'+subject+'*'))
        for file in files:
            dct_subc[file.split('/')[-1].split('.')[-3]] = pd.read_csv(file, header=None, squeeze=True)
        dfsubc = pd.DataFrame(dct_subc)

        ### Combime into one table
        dfbrain = pd.concat([dfcort, dfsubc], axis=1)
        #display(dfbrain)


        ### Load GLM matrix from FSL FEAT output
        mat_file = open('/media/data/HCPAging/data/'+task+'/data/'+subject+'/tfMRI_MSMAll_clean_fakeNIFTI.feat/design.mat', "r").read().split('\n')[5:]
        dct_mat={}
        for i in range(len(mat_file)):
            row = mat_file[i].split('\t')
            if (len(row) >= 3):
                x, y, _ = row
                dct_mat[i] =[float(x), float(y)]
            if (len(row) == 2): #for vismotor
                x, _ = row
                dct_mat[i] =[float(x)]
        mat = pd.DataFrame(dct_mat).T
        #display(mat)


        ### Clean signals from TASK and add highpass filterinf (0.008Hz)
        df_cleaned = pd.DataFrame(nls.clean(dfbrain.values, confounds=mat.values, t_r=0.8, detrend=False, high_pass=0.008),
                                  columns=dfbrain.columns)


        ### Calculate correlation matrix and make r-to-z transformation
        df_cleaned_corr_z = np.arctanh(df_cleaned.corr()) #with r-2-z


        ### Transform matrix to string for the subject
        dfg = df_cleaned_corr_z.where(np.triu(np.ones(df_cleaned_corr_z.shape), k=1).astype(bool)).stack().reset_index()
        indx = [i+'_&_'+j for i,j in zip(dfg['level_0'], dfg['level_1'])]
        fc_string = pd.Series(np.array(dfg[0]),index=indx)

        ### write result into group table
        dct_group_fc[subject] = fc_string

        ### save some intermediate results
        dfbrain.to_csv(path+subject+'/signalsROI/'+'task_brain_signals.csv')
        df_cleaned.to_csv(path+subject+'/signalsROI/'+'task_brain_signals_cleaned_from_task_hpass.csv')
        df_cleaned_corr_z.to_csv(path+subject+'/signalsROI/'+'task_brain_cleaned_hpass_corr_z.csv')

    #save group results
    pd.DataFrame(dct_group_fc).T.to_csv(('/'.join(path.split('/')[:-2]))+'/'+task.split('_')[-1]+'_task_FC.csv')

